# An analysis of tracks played by the Legacy Music Hour

First, we will pull all of the authorship data from the Blogger website.

In [5]:
import re
def lmh_standardize_whitespace(text):
    """ There is some nonuniformity in the episode descriptions of line breaks.
        some this routine standardizes the line breaks to simply be newlines
        while this breaks the html for display, it helps with parsing."""
    # first replace <br />\n -> <br />
    text = re.sub('<br\s*?/>\n', '<br />', text)
    # then replace <br /> -> \n
    return re.sub('<br\s*?/>', '\n', text)


In [6]:
from bs4 import BeautifulSoup
import urllib2
import json
import codecs

# store data in a temporary file (need to use codec because of weird chars in the input)
fout = codecs.open("lmh_data.txt",'w','utf-8')

# craft a query to pull the entire set of blogger entries
response = urllib2.urlopen('http://legacymusichour.blogspot.ca/search?updated-min=2011-01-01T00:00:00-08:00&updated-max=2015-08-01T00:00:00-08:00')

# pull the data
html = response.read().decode('utf-8')
response.close()

# parse the html
soup = BeautifulSoup(lmh_standardize_whitespace(html))
has_older = ( soup.find("a",class_="blog-pager-older-link") != None )
while has_older:
    
    for outer_p in soup.find_all("div",class_="post-outer"):
        title_text = outer_p.find("h3",class_="post-title entry-title")
        if title_text != None and "Episode" in title_text.text:
            episode_number = ''.join([c for c in title_text.text.strip().split(':')[0] if c.isdigit()])
            title_text.text.strip()
            rawlist = [i.strip().split(' - ') for i in outer_p.text.strip().split('\n\n') if ' - ' in i and len(i.split(' - '))>=6]
            if len(rawlist) > 0:
                for j in rawlist:
                    if j[1] != 'Composer':
                        fout.write(episode_number+';'+(';'.join(j))+'\n')

    has_older = ( soup.find("a",text="Older Posts") != None )
    if has_older:
        response = urllib2.urlopen(soup.find("a",text="Older Posts").get('href'))
        html = response.read().decode('utf-8')
        response.close()
        soup = BeautifulSoup(html)

fout.close()

### Cleaning and Normalizing the data

The above script pulls the data with some errors. We need first clean the entries.

In [ ]:
# first let's look at the episode numbers



